In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
import time
from gensim import corpora, models, similarities
path_to_data = "~/Documents/polytechnique/3A/nlp/link-prediction/data/"

In [2]:
divide_by = 100
sample_size_string = str(divide_by)

In [3]:
nodes_header = ["id", "year", "title", "authors", "journal", "abstract"]
nodes = pd.read_csv(path_to_data+"node_information.csv", names=nodes_header)
nodes.head()

,id,year,title,authors,journal,abstract
0,1001,2000,compactification geometry and duality,Paul S. Aspinwall,NaN,these are notes based on lectures given at tas...
1,1002,2000,domain walls and massive gauged supergravity p...,"M. Cvetic, H. Lu, C.N. Pope",Class.Quant.Grav.,we point out that massive gauged supergravity ...
2,1003,2000,comment on metric fluctuations in brane worlds,"Y.S. Myung, Gungwon Kang",NaN,recently ivanov and volovich hep-th 9912242 cl...
3,1004,2000,moving mirrors and thermodynamic paradoxes,Adam D. Helfer,Phys.Rev.,quantum fields responding to moving mirrors ha...
4,1005,2000,bundles of chiral blocks and boundary conditio...,"J. Fuchs, C. Schweigert",NaN,proceedings of lie iii clausthal july 1999 var...


In [4]:
names = ["id1", "id2", "target"]
training = pd.read_csv(path_to_data+"training_set.txt", names=names, delimiter=" ")
training_set_size = 615512
new_training_size = int(training_set_size/divide_by)

In [5]:
sample_1 = training.sample(frac=1.0/divide_by, replace=False)
sample_1.describe()

,id1,id2,target
count,6.155000e+03,6.155000e+03,6155.000000
mean,5.303932e+06,6.889071e+06,0.544110
std,4.754395e+06,4.301165e+06,0.498091
min,1.001000e+03,1.002000e+03,0.000000
25%,1.112450e+05,2.092545e+05,0.000000
50%,9.310049e+06,9.505105e+06,1.000000
75%,9.709110e+06,9.710089e+06,1.000000
max,9.912290e+06,9.912287e+06,1.000000


In [6]:
sample_2 = sample_1.copy()
sample_2.columns = ["id2", "id1", "target"]
sample_2.describe()

,id2,id1,target
count,6.155000e+03,6.155000e+03,6155.000000
mean,5.303932e+06,6.889071e+06,0.544110
std,4.754395e+06,4.301165e+06,0.498091
min,1.001000e+03,1.002000e+03,0.000000
25%,1.112450e+05,2.092545e+05,0.000000
50%,9.310049e+06,9.505105e+06,1.000000
75%,9.709110e+06,9.710089e+06,1.000000
max,9.912290e+06,9.912287e+06,1.000000


In [7]:
names = ["id1", "id2"]
testing = pd.read_csv(path_to_data+"testing_set.txt", names=names, delimiter=" ")
testing_set_size = len(testing)
new_testing_size = int(training_set_size/divide_by)

In [8]:
sample_1_testing = testing.sample(frac=1.0/divide_by, replace=False)
sample_1_testing.describe()

,id1,id2
count,3.260000e+02,3.260000e+02
mean,5.486836e+06,6.729696e+06
std,4.744125e+06,4.385983e+06
min,1.001000e+03,1.045000e+03
25%,2.010782e+05,2.049512e+05
50%,9.311584e+06,9.504122e+06
75%,9.710166e+06,9.710620e+06
max,9.912186e+06,9.912234e+06


In [9]:
sample_2_testing = sample_1_testing.copy()
sample_2_testing.columns = ["id2", "id1"]
sample_2_testing.describe()

,id2,id1
count,3.260000e+02,3.260000e+02
mean,5.486836e+06,6.729696e+06
std,4.744125e+06,4.385983e+06
min,1.001000e+03,1.045000e+03
25%,2.010782e+05,2.049512e+05
50%,9.311584e+06,9.504122e+06
75%,9.710166e+06,9.710620e+06
max,9.912186e+06,9.912234e+06


In [10]:
all_ids = pd.concat([sample_1, sample_2, sample_1_testing, sample_2_testing])
del all_ids["target"]
del all_ids["id2"]
all_ids.columns = ["id"]
all_ids.describe()

,id
count,1.296200e+04
mean,6.097093e+06
std,4.602116e+06
min,1.001000e+03
25%,2.032140e+05
50%,9.410236e+06
75%,9.709205e+06
max,9.912290e+06


In [11]:
all_ids_2 = all_ids.groupby(by="id").first().reset_index()

In [12]:
all_ids_2.describe()

,id
count,9.624000e+03
mean,5.992172e+06
std,4.622824e+06
min,1.001000e+03
25%,2.030705e+05
50%,9.406156e+06
75%,9.708038e+06
max,9.912290e+06


In [13]:
merged = all_ids_2.merge(right=nodes, how="inner")
merged.head()

,id,year,title,authors,journal,abstract
0,1001,2000,compactification geometry and duality,Paul S. Aspinwall,NaN,these are notes based on lectures given at tas...
1,1002,2000,domain walls and massive gauged supergravity p...,"M. Cvetic, H. Lu, C.N. Pope",Class.Quant.Grav.,we point out that massive gauged supergravity ...
2,1003,2000,comment on metric fluctuations in brane worlds,"Y.S. Myung, Gungwon Kang",NaN,recently ivanov and volovich hep-th 9912242 cl...
3,1004,2000,moving mirrors and thermodynamic paradoxes,Adam D. Helfer,Phys.Rev.,quantum fields responding to moving mirrors ha...
4,1005,2000,bundles of chiral blocks and boundary conditio...,"J. Fuchs, C. Schweigert",NaN,proceedings of lie iii clausthal july 1999 var...


In [14]:
merged.describe()

,id,year
count,9.624000e+03,9624.000000
mean,5.992172e+06,1998.193371
std,4.622824e+06,3.025149
min,1.001000e+03,1992.000000
25%,2.030705e+05,1996.000000
50%,9.406156e+06,1999.000000
75%,9.708038e+06,2001.000000
max,9.912290e+06,2003.000000


In [15]:
merged.to_csv(path_to_data+"node_information"+sample_size_string+".csv", header=False)

In [16]:
nodes = pd.read_csv(path_to_data+"node_information.csv", names=nodes_header)
nodes10 = pd.read_csv(path_to_data+"node_information"+sample_size_string+".csv", names=nodes_header)

In [17]:
nodes.dtypes

id           int64
year         int64
title       object
authors     object
journal     object
abstract    object
dtype: object

In [18]:
nodes10.dtypes

id           int64
year         int64
title       object
authors     object
journal     object
abstract    object
dtype: object

In [19]:
nodes.head()

,id,year,title,authors,journal,abstract
0,1001,2000,compactification geometry and duality,Paul S. Aspinwall,NaN,these are notes based on lectures given at tas...
1,1002,2000,domain walls and massive gauged supergravity p...,"M. Cvetic, H. Lu, C.N. Pope",Class.Quant.Grav.,we point out that massive gauged supergravity ...
2,1003,2000,comment on metric fluctuations in brane worlds,"Y.S. Myung, Gungwon Kang",NaN,recently ivanov and volovich hep-th 9912242 cl...
3,1004,2000,moving mirrors and thermodynamic paradoxes,Adam D. Helfer,Phys.Rev.,quantum fields responding to moving mirrors ha...
4,1005,2000,bundles of chiral blocks and boundary conditio...,"J. Fuchs, C. Schweigert",NaN,proceedings of lie iii clausthal july 1999 var...


In [20]:
nodes10.head()

,id,year,title,authors,journal,abstract
0,1001,2000,compactification geometry and duality,Paul S. Aspinwall,NaN,these are notes based on lectures given at tas...
1,1002,2000,domain walls and massive gauged supergravity p...,"M. Cvetic, H. Lu, C.N. Pope",Class.Quant.Grav.,we point out that massive gauged supergravity ...
2,1003,2000,comment on metric fluctuations in brane worlds,"Y.S. Myung, Gungwon Kang",NaN,recently ivanov and volovich hep-th 9912242 cl...
3,1004,2000,moving mirrors and thermodynamic paradoxes,Adam D. Helfer,Phys.Rev.,quantum fields responding to moving mirrors ha...
4,1005,2000,bundles of chiral blocks and boundary conditio...,"J. Fuchs, C. Schweigert",NaN,proceedings of lie iii clausthal july 1999 var...


In [21]:
nodes.describe()

,id,year
count,2.777000e+04,27770.000000
mean,6.096134e+06,1998.009039
std,4.581677e+06,3.124684
min,1.001000e+03,1992.000000
25%,2.041122e+05,1995.000000
50%,9.405182e+06,1998.000000
75%,9.705204e+06,2001.000000
max,9.912293e+06,2003.000000


In [22]:
nodes10.describe()

,id,year
count,9.624000e+03,9624.000000
mean,5.992172e+06,1998.193371
std,4.622824e+06,3.025149
min,1.001000e+03,1992.000000
25%,2.030705e+05,1996.000000
50%,9.406156e+06,1999.000000
75%,9.708038e+06,2001.000000
max,9.912290e+06,2003.000000


In [23]:
sample_1.to_csv(path_to_data+"training_set"+sample_size_string+".txt", header=False, sep=" ")
sample_1_testing.to_csv(path_to_data+"testing_set"+sample_size_string+".txt", header=False, sep=" ")